In [4]:
import re
from spellchecker import SpellChecker
from docx import Document
from docx.shared import Pt
import pyttsx3
import os
import comtypes.client
import speech_recognition as sr
from datetime import datetime

engine = pyttsx3.init()
engine.setProperty('rate', 130)
engine.setProperty('volume', 1.0)

word_save_path = 'filled_form.docx'
pdf_save_path = 'filled_form.pdf'

spell = SpellChecker()

field_type = {
    'Name': str, 'Age': int, 'Gender': str, 'Email': str, 'Number': int, 'Address': str
}

def speak_fields(text):
    engine.say(text)
    engine.runAndWait()

def notify_user_message():
    message = ("Hello, The form filling process is about to begin.\n"
               "Please listen carefully to the prompts.\n"
               "Speak only when 'Speak now' is displayed.\n"
               "If age is not recognized, please provide it as a sentence, like: 'I am 20 years old.'\n"
               "If you want to skip any field, just say 'Skip'.\n"
               "If you want to stop at any time, just say 'Stop'.\n"
               "If you want to repeat this message at any time, just say 'Repeat message'")
    print(message)
    speak_fields(message)

def repeat_message():
    repeat_message = ("Speak only when 'Speak now' is displayed.\n"
               "If age is not recognized, please provide it as a sentence, like: 'I am 20 years old.'\n"
               "If you want to repeat this message at any time, just say 'Repeat message'\n"
               "If you want to stop at any time, just say 'Stop'.")
    print(repeat_message)
    speak_fields(repeat_message)

def voice_input(prompt):
    recognizer = sr.Recognizer()
    voice = sr.Microphone()
    print(prompt)
    speak_fields(prompt)
    try:
        with voice as source:
            recognizer.adjust_for_ambient_noise(source)
            print('Speak now....')
            audio = recognizer.listen(source)
            print('Recognizing....')
            user_voice_input = recognizer.recognize_google(audio, language="en-IN")
            print(f'You said: {user_voice_input}')
            return user_voice_input.strip()
    except sr.UnknownValueError:
        print("I couldn't understand, please try again.")
        speak_fields("I couldn't understand, please try again.")
        return voice_input(prompt)

class AbortProcess(Exception):
    pass

def validate_input(user_input, field_name):
    expected_type = field_type.get(field_name, str)
    while True:
        if "skip" in user_input.lower():
            print(f"Skipping the field: {field_name}")
            speak_fields(f"Skipping the field: {field_name}")
            return ""
        elif "stop" in user_input.lower():
            message = ("Aborting the process as per your request...\n"
                       "Process Aborted.")
            raise AbortProcess(message)
        elif 'repeat message' in user_input.lower():
                print("Repeating message...")
                speak_fields("Repeating message")
                repeat_message()
                continue
        if field_name == "Gender":
            gender_input = user_input.strip().lower()
            if gender_input == 'female':
                return 'Female'
            elif gender_input in ["male", "mail"]:
                return "Male"
            else:
                print("Please enter a valid gender, either Male or Female.")
                speak_fields("Please enter a valid gender, either Male or Female.")
                user_input = voice_input(f"Enter {field_name}: ").strip()
        elif field_name == 'Age':
            numbers = re.findall(r'\b([1-9][0-9]?[0-9]?)\b', user_input)
            if numbers:
                age = int(numbers[0])
                if 1 <= age <= 999:
                    return age
                else:
                    print("Please enter a valid age number")
                    speak_fields("Please enter a valid age number")
            else:
                print("Age must be a numeric value or must include a valid age in the sentence.")
                speak_fields("Age must be a numeric value or must include a valid age in the sentence.")
            user_input = voice_input(f"Enter {field_name}: ").strip()
        elif field_name == 'Number':
            cleaned_number = ''.join(filter(str.isdigit, user_input))
            if len(cleaned_number) == 10:
                return cleaned_number
            else:
                print("Please enter a valid 10-digit phone number.")
                speak_fields("Please enter a valid 10-digit phone number.")
                user_input = voice_input(f"Enter {field_name}: ").strip()
        elif field_name == 'Email':
            email = r'^[A-Za-z0-9._]+@[A-Za-z0-9.]+\.[A-Za-z]{2,}$'
            if re.match(email, user_input):
                return user_input.lower()
            else:
                print("Please enter a valid email address.")
                speak_fields("Please enter a valid email address.")
                user_input = voice_input(f"Enter {field_name}: ").strip()
        elif expected_type == str:
            if all(word.isalpha() for word in user_input.split()):
                return " ".join(word.capitalize() for word in user_input.split())
            else:
                print(f"Please enter a valid {field_name}.")
                speak_fields(f"Please enter a valid {field_name}.")
                user_input = voice_input(f"Enter {field_name}: ").strip()
        elif expected_type == int:
            if user_input.isdigit():
                return int(user_input)
            else:
                print(f"Please enter a valid number for {field_name}.")
                speak_fields(f"Please enter a valid number for {field_name}.")
                user_input = voice_input(f"Enter {field_name}: ").strip()

def confirm_details(filled_form, fields_to_fill):
    while True:
        print("Please re-check the filled details and confirm if they are correct:")
        speak_fields("Please re-check the filled details and confirm if they are correct:")
        print(filled_form)
        confirmation = voice_input("Say 'Submit' to confirm and say 'No' or 'Incorrect' to re-enter details.").lower()
        if 'submit' in confirmation:
            print("Details confirmed, submitting the form.")
            speak_fields("Details confirmed, submitting the form.")
            return filled_form
        elif 'no' in confirmation or 'incorrect' in confirmation:
            print("Re-enter details...")
            speak_fields("Re-enter all details. Please provide the inputs again.")
            modified_form = re.sub(r": ____.*?_____", ": __", filled_form)
            return user_input(modified_form, fields_to_fill)
        else:
            print("Invalid input. Please say 'Submit' or 'No/Incorrect'.")
            speak_fields("Invalid input. Please say 'Submit' or 'No/Incorrect'.")

def user_input(fields_to_fill):
    filled_form = ""
    for field in fields_to_fill:
        while True:
            prompt = f'Please enter {field}'
            take_input = voice_input(prompt)
            take_input = validate_input(take_input, field)
            filled_form += f"{field}: _____{take_input}_____\n"
            break
    return confirm_details(filled_form, fields_to_fill)

def save_word(full_text, font_size=18):
    doc = Document()
    paragraph = doc.add_paragraph()
    for line in full_text.split('\n'):
        if ':' in line:
            field, value = line.split(': ', 1)
            run = paragraph.add_run(f'{field}: ')
            run.font.size = Pt(font_size)
            run = paragraph.add_run(value)
            run.font.size = Pt(font_size)
            run.font.underline = True
        else:
            run = paragraph.add_run(line)
            run.font.size = Pt(font_size)
        paragraph.add_run('\n')
    doc.save(word_save_path)
    print(f'Fields saved as {word_save_path}')

def word_to_pdf(word_path, pdf_save_path):
    word = comtypes.client.CreateObject('Word.Application')
    doc = word.Documents.Open(os.path.abspath(word_path))
    doc.SaveAs(os.path.abspath(pdf_save_path), FileFormat=17)
    doc.Close()
    word.Quit()
    print(f'Converted {word_path} to pdf: {pdf_save_path}')

try:
    notify_user_message()
    fields_to_fill = ['Name', 'Age', 'Gender']
    filled_form = user_input(fields_to_fill)
    if filled_form:
        save_word(filled_form, font_size=18)
        word_to_pdf(word_save_path, pdf_save_path)
except AbortProcess as e:
    print(str(e))
    speak_fields(str(e))
except FileNotFoundError:
    print('File not found')

Hello, The form filling process is about to begin.
Please listen carefully to the prompts.
Speak only when 'Speak now' is displayed.
If age is not recognized, please provide it as a sentence, like: 'I am 20 years old.'
If you want to skip any field, just say 'Skip'.
If you want to stop at any time, just say 'Stop'.
If you want to repeat this message at any time, just say 'Repeat message'
Please enter Name
Speak now....
Recognizing....
You said: samrudh dhondi
Please enter Age
Speak now....
Recognizing....
You said: 21
Please enter Gender
Speak now....
Recognizing....
You said: mail
Please re-check the filled details and confirm if they are correct:
Name: _____Samrudh Dhondi_____
Age: _____21_____
Gender: _____Male_____

Say 'Submit' to confirm and say 'No' or 'Incorrect' to re-enter details.
Speak now....
Recognizing....
You said: submit
Details confirmed, submitting the form.
Fields saved as filled_form.docx
Converted filled_form.docx to pdf: filled_form.pdf
